# PyAVD - Quick Solver

In [1]:
"""Modular aircraft concept"""
import pickle
import numpy as np
from gpkit import Model, Vectorize, parse_variables


class AircraftP(Model):
    """Aircraft flight physics: weight <= lift, fuel burn

    Variables
    ---------
    Wfuel  [lbf]  fuel weight
    Wburn  [lbf]  segment fuel burn

    Upper Unbounded
    ---------------
    Wburn, aircraft.wing.c, aircraft.wing.A

    Lower Unbounded
    ---------------
    Wfuel, aircraft.W, state.mu

    """
    @parse_variables(__doc__, globals())
    def setup(self, aircraft, state):
        self.aircraft = aircraft
        self.state = state

        self.wing_aero = aircraft.wing.dynamic(aircraft.wing, state)
        self.perf_models = [self.wing_aero]

        W = aircraft.W
        S = aircraft.wing.S

        V = state.V
        rho = state.rho

        D = self.wing_aero.D
        CL = self.wing_aero.CL

        return Wburn >= 0.1*D, W + Wfuel <= 0.5*rho*CL*S*V**2, {
            "performance":
                self.perf_models}


class Aircraft(Model):
    """The vehicle model

    Variables
    ---------
    W  [lbf]  weight

    Upper Unbounded
    ---------------
    W

    Lower Unbounded
    ---------------
    wing.c, wing.S
    """
    @parse_variables(__doc__, globals())
    def setup(self):
        self.fuse = Fuselage()
        self.wing = Wing()
        self.components = [self.fuse, self.wing]

        return [W >= sum(c.W for c in self.components),
                self.components]

    dynamic = AircraftP


class FlightState(Model):
    """Context for evaluating flight physics

    Variables
    ---------
    V     40       [knots]    true airspeed
    mu    1.628e-5 [N*s/m^2]  dynamic viscosity
    rho   0.74     [kg/m^3]   air density

    """
    @parse_variables(__doc__, globals())
    def setup(self):
        pass


class FlightSegment(Model):
    """Combines a context (flight state) and a component (the aircraft)

    Upper Unbounded
    ---------------
    Wburn, aircraft.wing.c, aircraft.wing.A

    Lower Unbounded
    ---------------
    Wfuel, aircraft.W

    """
    def setup(self, aircraft):
        self.aircraft = aircraft

        self.flightstate = FlightState()
        self.aircraftp = aircraft.dynamic(aircraft, self.flightstate)

        self.Wburn = self.aircraftp.Wburn
        self.Wfuel = self.aircraftp.Wfuel

        return {"aircraft performance": self.aircraftp,
                "flightstate": self.flightstate}


class Mission(Model):
    """A sequence of flight segments

    Upper Unbounded
    ---------------
    aircraft.wing.c, aircraft.wing.A

    Lower Unbounded
    ---------------
    aircraft.W
    """
    def setup(self, aircraft):
        self.aircraft = aircraft

        with Vectorize(4):  # four flight segments
            self.fs = FlightSegment(aircraft)

        Wburn = self.fs.aircraftp.Wburn
        Wfuel = self.fs.aircraftp.Wfuel
        self.takeoff_fuel = Wfuel[0]

        return {
            "fuel constraints":
                [Wfuel[:-1] >= Wfuel[1:] + Wburn[:-1],
                 Wfuel[-1] >= Wburn[-1]],
            "flight segment":
                self.fs}


class WingAero(Model):
    """Wing aerodynamics

    Variables
    ---------
    CD      [-]    drag coefficient
    CL      [-]    lift coefficient
    e   0.9 [-]    Oswald efficiency
    Re      [-]    Reynold's number
    D       [lbf]  drag force

    Upper Unbounded
    ---------------
    D, Re, wing.A, state.mu

    Lower Unbounded
    ---------------
    CL, wing.S, state.mu, state.rho, state.V
    """
    @parse_variables(__doc__, globals())
    def setup(self, wing, state):
        self.wing = wing
        self.state = state

        c = wing.c
        A = wing.A
        S = wing.S
        rho = state.rho
        V = state.V
        mu = state.mu

        return [D >= 0.5*rho*V**2*CD*S,
                Re == rho*V*c/mu,
                CD >= 0.074/Re**0.2 + CL**2/np.pi/A/e]


class Wing(Model):
    """Aircraft wing model

    Variables
    ---------
    W        [lbf]       weight
    S        [ft^2]      surface area
    rho    1 [lbf/ft^2]  areal density
    A     27 [-]         aspect ratio
    c        [ft]        mean chord

    Upper Unbounded
    ---------------
    W

    Lower Unbounded
    ---------------
    c, S
    """
    @parse_variables(__doc__, globals())
    def setup(self):
        return [c == (S/A)**0.5,
                W >= S*rho]

    dynamic = WingAero


class Fuselage(Model):
    """The thing that carries the fuel, engine, and payload

    A full model is left as an exercise for the reader.

    Variables
    ---------
    W  100 [lbf]  weight

    """
    @parse_variables(__doc__, globals())
    def setup(self):
        pass

AC = Aircraft()
MISSION = Mission(AC)
M = Model(MISSION.takeoff_fuel, [MISSION, AC])
print(M)


Cost Function
-------------
 Wfuel[0]

Constraints
-----------
 Mission
  "fuel constraints":
    Wfuel[:-1] >= Wfuel[1:] + Wburn[:-1]
    Wfuel[3] >= Wburn[3]

  FlightSegment
   AircraftP
    Wburn[:] >= 0.1*D[:]
    Aircraft.W + Wfuel[:] <= 0.5*rho[:]*CL[:]*S*V[:]^2
    "performance":
      WingAero
       D[:] >= 0.5*rho[:]*V[:]^2*CD[:]*S
       Re[:] = rho[:]*V[:]*c/mu[:]
       CD[:] >= 0.074/Re[:]^0.2 + CL[:]^2/PI/A/e[:]

   FlightState
    (no constraints)

 Aircraft
  Aircraft.W >= Aircraft.Fuselage.W + Aircraft.Wing.W
  Fuselage
   (no constraints)

  Wing
   c = (S/A)^0.5
   Aircraft.Wing.W >= S*Aircraft.Wing.rho


In [2]:
sol = M.solve(verbosity=0)
# # save solution to some files
# sol.savemat()
# sol.savecsv()
# sol.savetxt()
# sol.save("solution.pkl")
# # retrieve solution from a file
# sol_loaded = pickle.load(open("solution.pkl", "rb"))

vars_of_interest = set(AC.varkeys)
# # note that there's two ways to access submodels
# assert (MISSION["flight segment"]["aircraft performance"]
#         is MISSION.fs.aircraftp)
# vars_of_interest.update(MISSION.fs.aircraftp.unique_varkeys)
# vars_of_interest.add(M["D"])
print(sol.summary(vars_of_interest))
# print(sol.table(tables=["loose constraints"]))

# M.append(MISSION.fs.aircraftp.Wburn >= 0.2*MISSION.fs.aircraftp.wing_aero.D)
# sol = M.solve(verbosity=0)
# print(sol.diff("solution.pkl", showvars=vars_of_interest, sortbymodel=False))

try:
    from gpkit.interactive.sankey import Sankey
    variablesankey = Sankey(sol, M).diagram(AC.wing.A)
    sankey = Sankey(sol, M).diagram(width=1200, height=400, maxlinks=30)
    # the line below shows an interactive graph if run in jupyter notebook
    sankey  # pylint: disable=pointless-statement
except (ImportError, ModuleNotFoundError):
    print("Making Sankey diagrams requires the ipysankeywidget package")

from gpkit.interactive.references import referencesplot
referencesplot(M, openimmediately=False)


Optimal Cost
------------
 1.091

Free Variables
--------------
  | Aircraft.Wing
S : 44.14  [ft**2] surface area
W : 44.14  [lbf]   weight
c : 1.279  [ft]    mean chord

  | Aircraft
W : 144.1  [lbf]   weight

Variable Sensitivities
----------------------
    | Aircraft.Fuselage
  W : +0.97  weight

    | Aircraft.Wing
  A : -0.67  aspect ratio
rho : +0.43  areal density

Next Most Sensitive Variables
-----------------------------
    | Mission.FlightSegment.AircraftP.WingAero
  e : [ -0.18     -0.18     -0.18     -0.18    ] Oswald efficiency

    | Mission.FlightSegment.FlightState
  V : [ -0.22     -0.21     -0.21     -0.21    ] true airspeed
rho : [ -0.12     -0.11     -0.11     -0.11    ] air density

Most Sensitive Constraints
--------------------------
       | Aircraft
  +1.4 : .W >= .Fuselage.W + .Wing.W

       | Mission
    +1 : Wfuel[0] >= Wfuel[1] + Wburn[0]
 +0.75 : Wfuel[1] >= Wfuel[2] + Wburn[1]
  +0.5 : Wfuel[2] >= Wfuel[3] + Wburn[2]

       | Aircraft.Wing
 +0.43 : 

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Python\\Python39\\lib\\site-packages\\gpkit\\interactive\\referencesplot.html'

In [ ]:
sankey

SankeyWidget(layout=Layout(height='400', width='1200'), links=[{'source': 'Model.0001.0001.0001', 'target': 'M…